# Домашнее задание №2 
Выполнил Столяров Марк БЭК194

В данной работе вы будете изучать, как различные факторы влияют на зарплату женщин с учетом неслучайного отбора в число занятых. Для этого используется встроенный датафрейм, описание которого дано в файле “Homework1.R”. Для тех, кто не использует R, прикладывается excel файл “Mroz87.xlsx” с соответствующими данными.

In [1]:
%load_ext rpy2.ipython

In [2]:
import pandas as pd
import numpy as np

In [3]:
%%R

#install.packages("ggplot2", repos='http://cran.us.r-project.org', quiet=TRUE)
library(ggplot2)
library('sampleSelection')
library("GJRM")                             
library("pbivnorm")  
library("pROC")
library("DescTools")
library("glmx") 
library("numDeriv")
library("margins")
library("crch")                     
library("hpa")                           
library("readxl")
library("mvtnorm")


R[write to console]: Загрузка требуемого пакета: maxLik

R[write to console]: Загрузка требуемого пакета: miscTools

R[write to console]: 
Please cite the 'maxLik' package as:
Henningsen, Arne and Toomet, Ott (2011). maxLik: A package for maximum likelihood estimation in R. Computational Statistics 26(3), 443-458. DOI 10.1007/s00180-010-0217-1.

If you have questions, suggestions, or comments regarding the 'maxLik' package, please use a forum or 'tracker' at maxLik's R-Forge site:
https://r-forge.r-project.org/projects/maxlik/

R[write to console]: Загрузка требуемого пакета: mgcv

R[write to console]: Загрузка требуемого пакета: nlme

R[write to console]: This is mgcv 1.8-40. For overview type 'help("mgcv-package")'.

R[write to console]: 
This is GJRM 0.2-6.
For overview type 'help("GJRM-package")'.


R[write to console]: Type 'citation("pROC")' for a citation.

R[write to console]: 
Присоединяю пакет: ‘pROC’


R[write to console]: Следующие объекты скрыты от ‘package:stats’:

    co

In [4]:
%%R
help('Mroz87')

File: /var/folders/8x/3rynjhnd7016rpmk3k2d0k8c0000gn/T//RtmpzD2YC3/Rtxte5ac7a9593a1
Mroz87             package:sampleSelection             R Documentation



_U._S. _W_o_m_e_n'_s _L_a_b_o_r _F_o_r_c_e _P_a_r_t_i_c_i_p_a_t_i_o_n



_D_e_s_c_r_i_p_t_i_o_n:



     The ‘Mroz87’ data frame contains data about 753 married women.

     These data are collected within the "Panel Study of Income

     Dynamics" (PSID).  Of the 753 observations, the first 428 are for

     women with positive hours worked in 1975, while the remaining 325

     observations are for women who did not work for pay in 1975.  A

     more complete discussion of the data is found in Mroz (1987),

     Appendix 1.



_U_s_a_g_e:



     data(Mroz87)

     

_F_o_r_m_a_t:



     This data frame contains the following columns:



     lfp Dummy variable for labor-force participation.



     hours Wife's hours of work in 1975.



     kids5 Number of children 5 years

## Часть 1. Теория и гипотезы
$\bf{1.1.}$ Выберите независимые переменные для уравнения зарплаты и уравнения занятости. Кратко теоретически обоснуйте выбор каждой из них: не обязательно со ссылками на литературу, достаточно здравого смысла. Укажите и кратко обоснуйте предполагаемые направления эффектов. Уравнение занятости должно включать по крайней мере одну переменную, которой не было в уравнении зарплаты, и одну переменную,
которая есть в уравнении зарплаты. Желательно, чтобы общая для двух уравнений переменная была непрерывной, например, возраст, а также не использовать более трех переменных в каждом из уравнений.

$\bf{\text{Решение:}}$

Для уравнения заработной платы возьмем возраст (`age`) и кол-во лет образования (`educ`). С одной стороны, чем старше человек, тем он менее производительный (меньше вынослив, например), что может понижать заработную плату, с другой $-$ возраст может отражать стаж и, соответственно, навыки человека, а чем они выше, тем больше (вероятно) у человека будет заработная плата. Так что эффект неоднозначен. Уровень образования пропорционален человеческому капиталу, поэтому у работников с большим временем обучения (более высоким образованием) заработная плата, скорее всего, должна быть больше, так как такой человек способен выполнять более сложный труд.

Для уравнения занятости возьмем так же непрерывную переменную возраста (`age`). Чем старше человек, тем больше у него было времени для получения соответствующих навыков и для нахождения рабочего места (выигрыша конкуренции за рабочее место), что соответствует тому, что влияние возраста на занятость может быть положительным, хотя по достижению определенного возраста люди перестают работать (становятся пенсионерами), поэтому эффект опять же неоднозначен. Вторым регрессором в уравнении отбора будет выступать число детей у человека, которое мы сформируем как `kids = kids5 + kids618`. Чем больше у человека детей, тем больше ему необходимо денег, чтобы прокормить их, а для этого нужно зарабатывать, то есть, работать. Эффект кол-ва детей на уравнение занятости, кажется, должен быть положительным.

In [5]:
df = pd.read_excel("Mroz87.xlsx")
df['kids'] = df['kids5'] + df['kids618']
df.describe()

,lfp,hours,kids5,kids618,age,educ,wage,repwage,hushrs,husage,...,mtr,motheduc,fatheduc,unem,city,exper,nwifeinc,wifecoll,huscoll,kids
count,753.000000,753.000000,753.000000,753.000000,753.000000,753.000000,753.000000,753.000000,753.000000,753.000000,...,753.000000,753.000000,753.000000,753.000000,753.000000,753.00000,753.000000,753.000000,753.000000,753.000000
mean,0.568393,740.576361,0.237716,1.353254,42.537849,12.286853,2.374565,1.849734,2267.270916,45.120850,...,0.678863,9.250996,8.808765,8.623506,0.642762,10.63081,20.128964,0.281541,0.391766,1.590969
std,0.495630,871.314216,0.523959,1.319874,8.072574,2.280246,3.241829,2.419887,595.566649,8.058793,...,0.083496,3.367468,3.572290,3.114934,0.479504,8.06913,11.634797,0.450049,0.488469,1.460481
min,0.000000,0.000000,0.000000,0.000000,30.000000,5.000000,0.000000,0.000000,175.000000,30.000000,...,0.441500,0.000000,0.000000,3.000000,0.000000,0.00000,-0.029057,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,36.000000,12.000000,0.000000,0.000000,1928.000000,38.000000,...,0.621500,7.000000,7.000000,7.500000,0.000000,4.00000,13.025040,0.000000,0.000000,0.000000
50%,1.000000,288.000000,0.000000,1.000000,43.000000,12.000000,1.625000,0.000000,2164.000000,46.000000,...,0.691500,10.000000,7.000000,7.500000,1.000000,9.00000,17.700001,0.000000,0.000000,1.000000
75%,1.000000,1516.000000,0.000000,2.000000,49.000000,13.000000,3.787900,3.580000,2553.000000,52.000000,...,0.721500,12.000000,12.000000,11.000000,1.000000,15.00000,24.466000,1.000000,1.000000,3.000000
max,1.000000,4950.000000,3.000000,8.000000,60.000000,17.000000,25.000000,9.980000,5010.000000,60.000000,...,0.941500,17.000000,17.000000,14.000000,1.000000,45.00000,96.000000,1.000000,1.000000,8.000000


In [6]:
%%R

data <- read_excel("Mroz87.xlsx")
data$kids <- data$kids5 + data$kids618
data

# A tibble: 753 × 23
     lfp hours kids5 kids618   age  educ  wage repwage hushrs husage huseduc
   <dbl> <dbl> <dbl>   <dbl> <dbl> <dbl> <dbl>   <dbl>  <dbl>  <dbl>   <dbl>
 1     1  1610     1       0    32    12  3.35    2.65   2708     34      12
 2     1  1656     0       2    30    12  1.39    2.65   2310     30       9
 3     1  1980     1       3    35    12  4.55    4.04   3072     40      12
 4     1   456     0       3    34    12  1.10    3.25   1920     53      10
 5     1  1568     1       2    31    14  4.59    3.6    2000     32      12
 6     1  2032     0       0    54    12  4.74    4.7    1040     57      11
 7     1  1440     0       2    37    16  8.33    5.95   2670     37      12
 8     1  1020     0       0    54    12  7.84    9.98   4120     53       8
 9     1  1458     0       2    48    12  2.13    0      1995     52       4
10     1  1600     0       2    39    12  4.69    4.15   2100     43      12
# … with 743 more rows, and 12 more variables: huswage 

## Часть 2. Модель Тобина
$\bf{2.1.}$ Оцените Тобит модель, предварительно записав максимизируемую функцию правдоподобия. Результат представьте в форме таблицы (можно, например, использовать выдачу из stata, R или python).

$\bf{\text{Решение:}}$

Пусть $X\; -$ матрица регрессоров (векторы единиц, возраста и числа лет образования индивида), $y\;-$ наблюдаемая заработная плата (`wage`). Модель Тобина имеет следующий вид (в нашей задаче, где левая граница равна нулю):
$$y^*_i = x_i'\beta + \varepsilon_i, \text{ } \varepsilon_i \sim \mathrm{N}(0, \sigma^2)$$
$$
y_i = y^*_i \cdot I(y^*_i > 0)
$$
где $I(x)\; -$ индикатор того, что условие $x$ выполняется. То есть, в нашей задаче зарплаты не наблюдаются для тех, кто не работает (у них мы видим значение $y_i=0$).

Тогда функцию правдоподобия, что мы будем максимизировать для нахождения $\beta$ и $\sigma$, можно записать следующим образом:

$$
L(X, y, \beta, \sigma) = \prod_{y_i > 0}\frac{1}{\sigma} \phi\left(\frac{y_i - x'_i \beta}{\sigma}\right) \prod_{y_i = 0} P(y_i^* < 0) 
= \prod_{y_i > 0}\frac{1}{\sigma} \phi\left(\frac{y_i-x'_i \beta}{\sigma}\right) \prod_{y_i = 0} \Phi(-x'_i \beta)
\longrightarrow \max_{\beta, \sigma}
$$

Здесь $\Phi()$ и $\phi()$ $-$ соответственно, функции распределения и плотности стандартной нормальной случайной величины.

In [7]:
%%R

model_tb <- crch(wage ~ age + educ,      
                 data = data,                                
                 left = 0,                          
                 truncated = FALSE,                  
                 dist = "gaussian")   

summary(model_tb)  


Call:
crch(formula = wage ~ age + educ, data = data, dist = "gaussian", left = 0, 
    truncated = FALSE)

Standardized residuals:
    Min      1Q  Median      3Q     Max 
-0.9096 -0.1116  0.2725  0.6604  5.1755 

Coefficients (location model):
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -7.02410    1.51982  -4.622 3.81e-06 ***
age         -0.01668    0.02344  -0.712    0.477    
educ         0.69445    0.08370   8.297  < 2e-16 ***

Coefficients (scale model with log link):
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  1.54892    0.03726   41.57   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 

Distribution: gaussian
Log-likelihood: -1508 on 4 Df
Number of iterations in BFGS optimization: 18 


$\bf{2.2.}$ Опишите преимущества Тобит модели над усеченной регрессией. Объясните, в каких случаях можно использовать усеченную регрессию, но не получится использовать Тобит модель: приведите гипотетический (можно использовать фантазию) пример.

$\bf{\text{Решение:}}$

Тобит модель позволяет учитывать наблюдения, для которых известны регрессоры, но неизвестны значения зависимой переменной (известно, что для некоторых наблюдений таргет принимает граничное значение), усеченная регрессия же использует лишь наблюдения с известными как регрессорами, так и регрессантами. Поэтому оценки Тобит модели являются более эффективными в таком случае (когда целевая переменная может принимать, как непрерывные, так и дискретные граничные значения).

Соответственно, использование Тобита не представляется валидным, когда нам доступна лишь усеченная выборка (то есть, нет наблюдений, для которых известны факторы, но неизвестен таргет). Тогда Тобит даст оценки ММП для наблюдений с $y_i > 0$, что будет аналогично обычному МНК и никак не будет учитывать смещение в выборке, поэтому оценки получатся несостоятельными. Пример (усеченная выборка): для исследования зависимости дохода от разных социальных факторов в Москве были опрошены жители беднейших районов города (в результате богатые респонденты в выборку не попали, ни их доход, ни их признаки). 




$\bf{2.3.}$ Проинтерпретируйте полученные значения оценок для каждой независимой переменной. Поясните, как полученные результаты соотносятся с высказанными вами ранее предположениями.

$\bf{\text{Решение:}}$

С увеличением числа лет образования на 1 при прочих равных увеличивает почасовой доход на приблизительно 0.7$, что согласуется с нашими предположениями. Коэффициент при возрасте оказался незначимым, что может говорить о двунаправленном влиянии возраста на доход (такие предположения мы тоже высказывали). Возможно, включение возраста нелинейно изменило бы ситуацию (такой случай будет представлен и проинтерпретирован) ниже.

$\bf{2.4.}$ Для индивида с произвольными характеристиками укажите (предварительно записав используемые для расчетов формулы):
1. $E(y^*)$; 
2. $E(y)$;
3. Вероятности того, что индивид работает.

$\bf{\text{Решение:}}$

Возьмем произвольного индивида, которому 21 год и который проучился 14 лет. Пусть $x=(1, 21, 14)^T$.

In [8]:
%%R

Mark <- data.frame(age = 21,
                   educ = 14)

est_tb <- coef(model_tb)                             
coef_tb <- est_tb[-length(est_tb)]                                                                       
sigma_tb <- exp(est_tb[length(est_tb)]) 

1. $E(y^*)$.

$$E(y^*) = E(x' \beta + \varepsilon) = x' \beta \approx 2.35$$



In [9]:
%%R

wage_est <- predict(model_tb,
                    newdata = Mark)

wage_est

       1 
2.347898 


2. $E(y)$.

$$
E(y) = E(y | y^* > 0) \cdot P(y^* > 0) + E(0 | y^* < 0)\cdot P(y^* < 0) = (x'\beta + \sigma \lambda(\alpha)) \cdot \Phi\left(\frac{x'\beta}{\sigma}\right)\approx3.28
,
$$

где $\lambda(\alpha) = \frac{\phi(\alpha)}{1 - \Phi(\alpha)}
= \frac{\phi(\alpha)}{\Phi(-\alpha)}$, а $\alpha=\frac{0 - x'\beta}{\sigma}$.

In [10]:
%%R

prob_work <- pnorm(wage_est / sigma_tb)

a <- (0 - wage_est) / sigma_tb
lambda <- dnorm(a) / pnorm(-a)
epsilon_E <- sigma_tb * lambda

wage_cond <- wage_est + epsilon_E
wage_est_cens <- prob_work * wage_cond
wage_est_cens

       1 
3.280432 


3. Вероятность того, что индивид работает.

$$
P(y^* > 0) = P(x'\beta + \varepsilon > 0) = 1 - \Phi\left(-\frac{x'\beta}{\sigma}\right) = \Phi\left( \frac{x'\beta}{\sigma}\right) \approx 0.69
$$

In [11]:
%%R

prob_est <- pnorm(wage_est / sigma_tb)
prob_est

        1 
0.6910672 


$\bf{2.5.}$ Для индивида с произвольными характеристиками рассчитайте предельный эффект любой переменной (не дамми), входящей линейно (предварительно записав используемые для расчетов формулы) на:
1. $E(y^*)$;
2. $E(y)$;
3. Вероятности того, что индивид работает

$\bf{\text{Решение:}}$

Найдем предельные эффекты возраста ($x_{s}$) на указанные величины для индивида $i$, характеристики которого указаны в предыдущем пункте:

1. $E(y^*)$.

$$\frac{\partial E(y^*)}{\partial x_s} = \beta_s \approx -0.017$$

In [12]:
%%R

coef_tb[2]

        age 
-0.01668404 


2. $E(y)$.

$$
E(y) = \Phi\left(\frac{x'\beta}{\sigma}\right)\cdot x'\beta + \sigma \phi\left(\frac{x'\beta}{\sigma} \right)
$$


$$\frac{\partial E(y)}{\partial x_s} = \frac{\beta_s}{\sigma}\cdot \phi\left(\frac{x'\beta}{\sigma} \right)\cdot x'\beta + \beta_s \Phi\left(\frac{x'\beta}{\sigma} \right) + \sigma\cdot \frac{\partial\phi\left(\frac{x'\beta}{\sigma}\right)}{\partial x_s}$$

Заметим, что 
$$
\frac{\partial\phi\left(\frac{x'\beta}{\sigma}\right)}{\partial x_s} = \phi\left(\frac{x'\beta}{\sigma}\right) \cdot \frac{-2\beta_s \cdot x'\beta}{2\sigma^2}
$$

Тогда
$$\frac{\partial E(y)}{\partial x_s} = \beta_s \Phi\left(\frac{x'\beta}{\sigma} \right)\approx-0.0115$$


In [13]:
%%R
ME_cens <- coef_tb[2] * prob_est
ME_cens

        age 
-0.01152979 


3. Вероятность того, что индивид работает.

$$P(y^* > 0) = \Phi\left( \frac{x'\beta}{\sigma}\right)$$

$$
\frac{\partial P(y^* > 0)}{\partial x_s} = \frac{\beta_s}{\sigma}\cdot \phi\left(\frac{x'\beta}{\sigma} \right)
\approx -0.00125
$$

In [14]:
%%R

ME_prob <- coef_tb[2] / sigma_tb * dnorm(wage_est / sigma_tb)
ME_prob

        age 
-0.00124877 


$\bf{2.6^*.}$ Добавьте в модель нелинейный эффект (например, квадрат). Повторите предыдущий пункт для переменной, имеющей нелинейный эффект.

$\bf{\text{Решение:}}$

Добавим в модель возраст в квадрате (по-прежнему для краткости будем обозначать переменную возраста как $x_s$, а кэффициенты при возрасте и возрасте в квадрате как $\beta_s$ и $\beta_{s2}, соответственно$).

In [15]:
%%R

model_tb_age2 <- crch(wage ~ age + I(age ^ 2) + educ,      
                 data = data,                                
                 left = 0,                          
                 truncated = FALSE,                  
                 dist = "gaussian")   

summary(model_tb_age2)  


Call:
crch(formula = wage ~ age + I(age^2) + educ, data = data, dist = "gaussian", 
    left = 0, truncated = FALSE)

Standardized residuals:
    Min      1Q  Median      3Q     Max 
-0.9411 -0.1032  0.2445  0.6701  5.1553 

Coefficients (location model):
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -18.711175   5.562244  -3.364 0.000768 ***
age           0.544455   0.257260   2.116 0.034314 *  
I(age^2)     -0.006519   0.002978  -2.189 0.028597 *  
educ          0.697209   0.083617   8.338  < 2e-16 ***

Coefficients (scale model with log link):
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  1.54605    0.03725   41.51   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 

Distribution: gaussian
Log-likelihood: -1505 on 5 Df
Number of iterations in BFGS optimization: 25 


Заметим, что теперь коэффициенты при возрасте стали значимыми на 5%-ом уровне. Значимая параболическая зависимость дохода от возраста, при том, парабола с ветвями вниз и вершиной при возрасте равном 45. То есть, сперва по мере взросления, когда человек набирается опыта, его доход с каждым новым годом увеличивается при прочих равных, а после 45, когда он стареет и теряет в своей производительности, его доход с каждым годом уменьшается.

In [16]:
%%R

est_tb_age2 <- coef(model_tb_age2)                             
coef_tb_age2 <- est_tb_age2[-length(est_tb_age2)]                                                                       
sigma_tb_age2 <- exp(est_tb_age2[length(est_tb_age2)])

Теперь найдем предельные эффекты:

1. $E(y^*)$.

$$\frac{\partial E(y^*)}{\partial x_s} = \beta_s + 2 \beta_{2s}x_s \approx 0.27$$


In [17]:
%%R

est_tb_age2[2] + 2 * est_tb_age2[3] * Mark[1]

        age
1 0.2706382


2. $E(y)$.

$$
E(y) = \Phi\left(\frac{x'\beta}{\sigma}\right)\cdot x'\beta + \sigma \phi\left(\frac{x'\beta}{\sigma} \right)
$$


$$\frac{\partial E(y)}{\partial x_s} = \frac{\beta_s + 2 \beta_{2s}x_s}{\sigma}\cdot \phi\left(\frac{x'\beta}{\sigma} \right)\cdot x'\beta
+ (\beta_s + 2 \beta_{2s}x_s)\cdot \Phi\left(\frac{x'\beta}{\sigma} \right) 
+ \sigma\cdot \frac{\partial\phi\left(\frac{x'\beta}{\sigma}\right)}{\partial x_s}$$

Заметим, что 
$$
\frac{\partial\phi\left(\frac{x'\beta}{\sigma}\right)}{\partial x_s} = \phi\left(\frac{x'\beta}{\sigma}\right) \cdot \frac{-2(\beta_s + 2 \beta_{2s}x_s) \cdot x'\beta}{2\sigma^2}
$$

Тогда
$$\frac{\partial E(y)}{\partial x_s} = (\beta_s + 2 \beta_{2s}x_s)\cdot \Phi\left(\frac{x'\beta}{\sigma} \right) \approx 0.126$$



In [18]:
%%R

(est_tb_age2[2] + 2 * est_tb_age2[3] * Mark[1]) * pnorm(predict(model_tb_age2, 
                     newdata = Mark) / sigma_tb_age2)

        age
1 0.1263161


3. Вероятность того, что индивид работает.

$$P(y^* > 0) = \Phi\left( \frac{x'\beta}{\sigma}\right)$$

$$
\frac{\partial P(y^* > 0)}{\partial x_s} = \frac{\beta_s + 2 \beta_{2s}x_s}{\sigma}\cdot \phi\left(\frac{x'\beta}{\sigma} \right) \approx 0.023
$$

In [19]:
%%R

(est_tb_age2[2] + 2 * est_tb_age2[3] * Mark[1]) / sigma_tb_age2 * dnorm(predict(model_tb_age2, 
                     newdata = Mark) / sigma_tb_age2)

        age
1 0.0229269


$\bf\text{2.7**.}$ При помощи LR-теста проверьте гипотезу о гомоскедастичности в Тобит модели, предварительно формально записав предполагаемые нулевой гипотезой ограничения на параметры, асимптотическое распределение тестовой статистики (при верной нулевой гипотезе) и максимизируемую в гетероскедастичной Тобит модели функцию правдоподобия. При этом уравнение дисперсии должно включать по крайней мере одну переменную, не входящую в основное уравнение. Укажите негативные последствия, к которым может приводить отсутствие учета гетероскедастичности при условии ее наличия в Тобит модели.

$\bf{\text{Решение:}}$

Предположим, что в модели Тобита распределена следующим образом (то есть, дисперсия случайной ошибки зависит от возраста и кол-ва детей (гетероскедастичность)):

$$\varepsilon_i \sim N(0, (\exp(\tau_0 + \tau_1 age_i + \tau_2 kids_i))^2)$$



С таким распределением ошибки меняется функция правдоподобия и могут существенно измениться оценки коэффициентов $\beta$ (ввиду учтенной теперь гетероскедастичности). Это наша полная модель. 

Проведем LR-тест для проверки гетероскедастичности:

$$ H_0: \tau_2=\tau_1=0$$

Тестовая статистика
$$ LR = -2(l_R(\hat{\beta}_R) - l_{UR}(\hat{\beta}_{UR})) $$

имеет распределение $\chi^2(2)$ при верной нулевой гипотезе, так как у нас есть два ограничения. Ограниченной моделью выступает наша обычная Тобит модель, оцененнная выше.

Оценим модель с учетом гетероскедастичности:


In [20]:
%%R

model_htobit <- crch(wage ~ age + educ |       
                            age + kids,   
                     data = data,                                
                     left = 0,                          
                     link.scale = "log")    
summary(model_htobit) 


Call:
crch(formula = wage ~ age + educ | age + kids, data = data, link.scale = "log", 
    left = 0)

Standardized residuals:
    Min      1Q  Median      3Q     Max 
-1.0531 -0.1140  0.2536  0.6602  5.1791 

Coefficients (location model):
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -6.93970    1.52248  -4.558 5.16e-06 ***
age         -0.02463    0.02400  -1.026    0.305    
educ         0.71830    0.08438   8.512  < 2e-16 ***

Coefficients (scale model with log link):
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) 1.115962   0.225226   4.955 7.24e-07 ***
age         0.008647   0.004851   1.782   0.0747 .  
kids        0.042109   0.026181   1.608   0.1078    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 

Distribution: gaussian
Log-likelihood: -1506 on 6 Df
Number of iterations in BFGS optimization: 14 


Проведем LR-тест:

In [21]:
%%R

l_ur <- logLik(model_htobit)
l_r <- logLik(model_tb)

LR_ratio <- 2 * (l_ur - l_r)
p.value <- as.numeric(1 - pchisq(LR_ratio, df=2))  
p.value

[1] 0.121678


Так как p-value теста достаточно велико $(>0.1)$, у нас нет оснований отвергнуть нулевую гипотезу о гомоскедастичности.

Наличие гетероскедастичности в модели (не учтенной) могло бы привести к несостоятельным и неэффективным оценкам коэффициентов ММП.

$\bf2.8.^{***}$ Для индивида с произвольными характеристиками в Тобит модели с гетероскедастичной случайной ошибкой рассчитайте (предварительно записав соответствующую формулу) предельный эффект переменной (не дамми), входящей и в основное уравнение, и в уравнение дисперсии, на:
1. $E(y^*)$;
2. $E(y)$;
3. Вероятности того, что индивид работает.

$\bf{\text{Решение:}}$

Как и в пункте выше будем полагать, что ошибка распределенав следующим образом:
$$\varepsilon_i \sim N(0, (\exp(\tau_0 + \tau_1 age_i + \tau_2 kids_i))^2)$$

Найдем предельные эффекты по возрасту, который входит как в основное уравнение, так и в уравнение дисперсии:

1. $E(y^*)$. Здесь ничего не поменялось:

$$\frac{\partial E(y^*)}{\partial x_s} = \beta_s \approx -0.02463$$



In [22]:
%%R
est_tb_h <- coef(model_htobit)                             
coef_tb_h <- est_tb_h[-length(est_tb_h)] 
coef_tb_h[2]

        age 
-0.02462506 


2. $E(y)$.

Пусть $z = (1, age, kids)$. Тогда  $\sigma(z)=e^{z'\tau}$. Пусть $age_i=x_s$, $age_i$ есть и в $x_i$, и в $z_i$ (здесь, как везде раньше и после индексация индвида опускается):

$$
E(y) = \Phi\left(\frac{x'\beta}{\sigma(z)}\right)\cdot x'\beta + \sigma(z) \phi\left(\frac{x'\beta}{\sigma(z)} \right)
$$


$$
\frac{\partial\left( \frac{x'\beta}{\sigma(z)}\right)}{\partial x_s} = \frac{\beta_s \sigma(z) - x'\beta \cdot\sigma(z)\tau_s}{\sigma(z)^2} = \frac{\beta_s-x'\beta \cdot\tau_s}{\sigma(z)}
$$

$$\frac{\partial E(y)}{\partial x_s} = \frac{\beta_s-x'\beta \cdot\tau_s}{\sigma(z)}\cdot \phi\left(\frac{x'\beta}{\sigma(z)} \right)\cdot x'\beta + \beta_s \Phi\left(\frac{x'\beta}{\sigma(z)} \right) + \sigma(z)\cdot \frac{\partial\phi\left(\frac{x'\beta}{\sigma(z)}\right)}{\partial x_s}$$

Заметим, что 
$$
\frac{\partial\phi\left(\frac{x'\beta}{\sigma(z)}\right)}{\partial x_s} = \frac{\partial\phi\left(\frac{x'\beta}{\sigma(z)}\right)}{\partial \frac{x'\beta}{\sigma(z)}}\cdot \frac{\partial\frac{x'\beta}{\sigma(z)}}{\partial x_s} = -\frac{x'\beta}{\sigma(z)}\cdot\phi\left(\frac{x'\beta}{\sigma(z)}\right)\cdot \frac{\beta_s-x'\beta \cdot\tau_s}{\sigma(z)}$$

Тогда
$$\frac{\partial E(y)}{\partial x_s} = \beta_s \Phi\left(\frac{x'\beta}{\sigma(z)} \right)\approx-0.0187$$

С точки зрения выражения изменения опять же незначительны. Здесь и далее будем все считать для индивида с теми же характеристиками, у которого нет детей.

In [23]:
%%R

Mark <- data.frame(age = 21,
                   educ = 14, kids=0)
wage_pred_h <- predict(model_htobit, newdata=Mark)
sigma_Mark <- exp(est_tb_h[4] + est_tb_h[5] * 21 + est_tb_h[6] * 0)

est_tb_h[2] * pnorm(wage_pred_h / sigma_Mark)

        age 
-0.01874443 


3. Вероятность того, что индивид работает.

$$P(y^* > 0) = \Phi\left( \frac{x'\beta}{\sigma(z)}\right)$$

$$
\frac{\partial P(y^* > 0)}{\partial x_s} = \frac{\beta_s-x'\beta \cdot\tau_s}{\sigma(z)}\cdot \phi\left(\frac{x'\beta}{\sigma(z)} \right)
\approx -0.00399
$$

In [24]:
%%R

(est_tb_h[2] - wage_pred_h * est_tb_h[5]) / sigma_Mark * dnorm(wage_pred_h / sigma_Mark)

         age 
-0.003989467 


## Часть 3. Модель Хекмана
$\bf\text{3.1.}$ Оцените модель Хекмана с помощью метода максимального правдоподобия, предварительно записав максимизируемую функцию правдоподобия и указав независимые переменные в уравнении занятости, которое должно иметь по крайней мере одну переменную, не входящую в уравнение зарплаты. Результат представьте в форме таблицы (можно, например, использовать выдачу из stata, R или python).

$\bf{\text{Решение:}}$

Пусть $X$ $-$ матрица признаков из единиц, возраста и числа лет обучения (1, `age`, `educ`), $y$ $-$ вектор значений наблюдаемой заработной платы `wage`, $W$ $-$ матрица признаков из единиц, числа детей и возраста (1, `kids`, `age`), $z$ $-$ вектор значений занятости `lfp`. 

Модель Хекмана устроена следующим образом:
$$y^*_i = x_i'\beta + \varepsilon_i$$
$$z_i = I(w_i'\gamma + u_i > 0) \; - \text{уравнение занятости}$$
$$
y_i = y^*_i \cdot I(z_i = 1)\; - \text{уравнение заработной платы ($y_i=0$ при $z_i = 0$)}
$$

где вектор ($\varepsilon_i$, $u_i$) имеет двумерное нормальное распределение, $E(\varepsilon_i, u_i) = (0, 0)$, $Var(\varepsilon_i) = \sigma^2$, $Var(u_i) = 1$, $cov(\varepsilon_i, u_i) = cov(u_i, \varepsilon_i) = \rho\sigma$.



Значит, функция правдоподобия для максимизации выглядит следующим образом:

$$
L(X, W, y, z, \sigma, \rho, \gamma, \beta) = \prod_{z_i=0} P(z_i=0) \prod_{z_i=1} f(y_i, z_i=1) = 
$$

$$
= \prod_{z_i=0} \left( 1-\Phi\left(w_i'\gamma\right)\right) \prod_{z_i=1} \Phi\left(
\frac{\sigma w_i'\gamma + \rho(y_i - x_i'\beta)}{\sigma\sqrt{1- \rho^2}}
\right) \cdot \left(
\frac{1}{\sqrt{2\pi}\sigma}\exp\left(-\frac{(y_i - x_i'\beta)^2}{2\sigma^2}\right)
\right)
\longrightarrow \max_{\beta, \gamma, \sigma, \rho}
$$

In [25]:
%%R

model_mle <- selection(                              
  selection = lfp ~ kids + age,                    
  outcome = wage ~ age + educ,                  
  data = data,                                         
  method = "ml")                                    
summary(model_mle)                                  

--------------------------------------------
Tobit 2 model (sample selection model)
Maximum Likelihood estimation
Newton-Raphson maximisation, 4 iterations
Return code 8: successive function values within relative tolerance limit (reltol)
Log-Likelihood: -1596.995 
753 observations (325 censored and 428 observed)
8 free parameters (df = 745)
Probit selection equation:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.501422   0.325667   4.610 4.73e-06 ***
kids        -0.142156   0.037187  -3.823 0.000143 ***
age         -0.025843   0.006728  -3.841 0.000133 ***
Outcome equation:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -2.93917    1.23099  -2.388   0.0172 *  
age          0.02210    0.02005   1.102   0.2708    
educ         0.49826    0.06584   7.567 1.12e-13 ***
   Error terms:
      Estimate Std. Error t value Pr(>|t|)    
sigma  3.10580    0.10814  28.720   <2e-16 ***
rho   -0.05611    0.19492  -0.288    0.774    
---
Signif. codes:  0 ‘***

In [26]:
%%R
coef_mle <- coef(model_mle, part = "outcome")
sigma_mle <- model_mle$estimate["sigma"]  
rho_mle <- model_mle$estimate["rho"]   

$\bf3.2.$ Опишите отличия модели Хекмана от модели Тобина.

$\bf{\text{Решение:}}$

Обе модели были описаны нами выше. Модель Хекмана является более сложной относительно модели Тобина: в ней есть отдельно уравнение отбора, отдельно основное уравнение. В модели Тобина такой гибкости нет: за отбор отвечает то же самое, что и за (в нашем случае) заработную плату. Тобит $-$ частный случай Хекита, когда $\rho=1$, $\gamma\sigma=\beta$, $W=X$.

$\bf3.3.$ Воспользуйтесь методом Хекмана, основанным на двухшаговой процедуре и сравните оценки, с полученными с использованием метода Хекмана, основанном на методе максимального правдоподобия. Опишите относительные преимущества и недостатки обоих методов.

$\bf{\text{Решение:}}$

In [27]:
%%R

model_2st <- selection(                              
  selection = lfp ~ kids + age,                    
  outcome = wage ~ age + educ,                  
  data = data,                                        
  method = "2step")                                  
summary(model_2st)                                   

--------------------------------------------
Tobit 2 model (sample selection model)
2-step Heckman / heckit estimation
753 observations (325 censored and 428 observed)
9 free parameters (df = 745)
Probit selection equation:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.501177   0.325987   4.605 4.85e-06 ***
kids        -0.141963   0.037245  -3.812 0.000149 ***
age         -0.025843   0.006732  -3.839 0.000134 ***
Outcome equation:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -2.74649    1.33958  -2.050   0.0407 *  
age          0.02587    0.02259   1.145   0.2525    
educ         0.49643    0.06601   7.521 1.57e-13 ***
Multiple R-Squared:0.1197,	Adjusted R-Squared:0.1135
   Error terms:
              Estimate Std. Error t value Pr(>|t|)
invMillsRatio  -0.6584     1.4431  -0.456    0.648
sigma           3.1430         NA      NA       NA
rho            -0.2095         NA      NA       NA
--------------------------------------------


In [28]:
%%R

coef_2st <- coef(model_2st, part = "outcome")        
coef_2st <- coef_2st[-length(coef_2st)]              
rho_2st <- model_2st$rho  

In [29]:
%%R

data.frame("Heckman MLE" = coef_mle,                 
           "Heckman 2step" = coef_2st)   

            Heckman.MLE Heckman.2step
(Intercept)  -2.9391697   -2.74649310
age           0.0220963    0.02586941
educ          0.4982631    0.49643489


Оценки коэффициентов очень близки друг к другу (как и должно быть).

In [30]:
%%R

data.frame("Heckman MLE" = rho_mle,   
           "Heckman 2step" = rho_2st) 

    Heckman.MLE Heckman.2step
rho -0.05611249    -0.2094795


Оценки $\rho$ получились не очень похожими, но в обеих моделях они незначимы.

Оценки ММП подхода (при верности допущения о нормальном распределении ошибок) более эффективны и обладают всеми другими свойствами ММП оценок (состоятельность, асимптотическая нормальность, инвариантность...). Этот подход является сложным с точки зрения вычислений и работает некорректно при нарушении допущения о нормальном распределении случайных ошибок (оценки становятся неэффективными и несостоятельными). Также нет гарантий того, что у максимизируемой функции будет единственный максимум.

Оценки двухшаговой процедуры менее эффективны, чем ММП, однако вычислительно их получить гораздо проще. Кроме того, эта процедура оказывается более устойчива к нарушению допущения о нормальном распределении случайных ошибок (однако в таком случае все равно лучше использовать непараметрические/полупараметрические методы, потому что оценки все равно могут быть в таком случае несостоятельными). Также двухшаговый подход требует наличия ограничений исключения (exclusion restrictions), иначе на втором шаге возникнет сильная мультиколлинеарность и оценки будут неэффективными.



$\bf3.4.$ Проинтерпретируйте значимость и значение оценки корреляции между случайными ошибками в обоих оцененных моделях. Укажите, можно ли было бы обойтись оцениванием обычной МНК модели.

$\bf{\text{Решение:}}$

В обеих моделях оценки $\rho$ получились незначимыми (в модели с ММП p-value у $\rho$ больше 0.75, в модели с двухшаговой процедурой соответствующее p-value не указано, но можно посмотреть на p-value при обратном отношении Миллса, оно тоже очень большое), поэтому интерпретировать значение оценки корреляции не имеет смысла. Так как у нас нет статистических доказательств того, что $\rho$ отлично от нуля, то можно было бы обойтись оцениванием обычной МНК модели.

$\bf3.5.$ В любой из двух оцененных в данном разделе моделей для индивида с произвольными характеристиками рассчитайте (предварительно записав формулу):
1. $E(y^*|z = 1)$ и $E(y^*|z = 0)$;
2. предельный эффект любой переменной (не дамми), входящей линейно и в основное уравнение, и в уравнение занятости, на $E(y^*| z = 1)$ и $E(y^*| z = 0)$.

$\bf{\text{Решение:}}$

Рассчитаем нужные значения для индивида 21 года, не имеющего детей, который учился 14 лет. Используем модель с ММП.

1. $E(y^*|z = 1)$ и $E(y^*|z = 0)$. Используем [Википедию](https://en.wikipedia.org/wiki/Truncated_normal_distribution) для нахождения математического ожидания усеченного нормального распределения:

$$
E(y_i^*|z_i = 1) = x_i'\beta + \rho \sigma \cdot \frac{\phi\left(z_i' \gamma\right)}{\Phi\left(z_i' \gamma\right)}
\approx 4.45,
$$

$$
E(y_i^*|z_i = 0) = x_i'\beta - \rho \sigma \cdot \frac{\phi\left(z_i' \gamma\right)}{\Phi\left(-z_i' \gamma\right)}
\approx 4.76
$$

In [31]:
%%R

Mark <- data.frame(age = 21,
                   educ = 14, 
                   kids=0)

wage_star <- predict(model_mle, 
                     newdata = Mark, 
                     part = "outcome",                 
                     type = "unconditional") 

# оценим линейный индекс
work_li <- predict(model_mle, 
                   newdata = Mark, 
                   part = "selection",                   
                   type = "link")                      

# Оценим E(y*|z) вручную:
lambda_est_1 <- dnorm(work_li) / pnorm(work_li)         
lambda_est_2 <- dnorm(work_li) / pnorm(-work_li)
wage_star + rho_mle * model_mle$estimate["sigma"] * lambda_est_1           # E(y*|z = 1)

       1 
4.447706 


In [32]:
%%R
wage_star - rho_mle * model_mle$estimate["sigma"] * lambda_est_2           # E(y*|z = 0)

       1 
4.760582 


Проверка:

In [33]:
%%R

predict(model_mle, newdata = Mark, 
                     part = "outcome",               
                     type = "conditional")

E[yo|ys=0] E[yo|ys=1] 
  4.760582   4.447706 


Все верно!

2. Предельный эффект любой переменной (не дамми), входящей линейно и в основное уравнение, и в уравнение занятости, на $E(y^*| z = 1)$ и $E(y^*| z = 0)$. Найдем соответствующие предельные эффекты для возраста (обозначим этот фактор за $x_s$):

$$
\frac{\partial E(y_i^*|z_i = 1)}{\partial x_s} = \beta_s + \rho \sigma \cdot \left( \frac{\phi\left(z_i' \gamma\right)}{\Phi\left(z_i' \gamma\right)}\right)'_{x_s} = \beta_s + \rho \sigma \cdot \frac{\partial\lambda_1}{\partial x_s}
$$

Из 2.5.2:
$$
\frac{\partial \phi\left(z_i' \gamma\right)}{\partial x_s} = -\gamma_s z'\gamma \cdot \phi\left(z_i' \gamma\right)
$$
Также
$$
\frac{\partial \Phi\left(z_i' \gamma\right)}{\partial x_s} = \gamma_s \cdot \phi\left(z_i' \gamma\right)
$$

Тогда
$$
\frac{\partial\lambda_1}{\partial x_s} = \frac{-\gamma_s z'\gamma \cdot \phi\left(z_i' \gamma\right)\cdot\Phi\left(z_i' \gamma\right) - \phi\left(z_i' \gamma\right)\cdot \gamma_s \cdot \phi\left(z_i' \gamma\right)}
{\Phi\left(z_i' \gamma\right)^2} = - \gamma_s\left( z'\gamma \cdot \lambda_1 + \lambda_1^2 \right)
$$

Значит 
$$
\frac{\partial E(y_i^*|z_i = 1)}{\partial x_s} =
\beta_s - \rho \sigma \cdot  \gamma_s\left( z'\gamma \cdot \lambda_1 + \lambda_1^2 \right) \approx 0.0204
$$

In [34]:
%%R
coef_s_est <- coef(model_mle)[1:3]

age_ME1 <- coef_mle["age"] - rho_mle * sigma_mle *
                                (work_li * lambda_est_1 +
                                 lambda_est_1 ^ 2) *
                                coef_s_est["age"]
age_ME1

       age 
0.02037349 


Теперь для случая, когда $z_i=0$:
$$
E(y_i^*|z_i = 0) = x_i'\beta - \rho \sigma \cdot \frac{\phi\left(z_i' \gamma\right)}{\Phi\left(-z_i' \gamma\right)}
$$

$$
\frac{\partial E(y_i^*|z_i = 0)}{\partial x_s} = \beta_s - \rho \sigma \cdot \left( \frac{\phi\left(z_i' \gamma\right)}{\Phi\left(-z_i' \gamma\right)}\right)'_{x_s} = \beta_s - \rho \sigma \cdot \frac{\partial\lambda_2}{\partial x_s}
$$

Из 2.5.2:
$$
\frac{\partial \phi\left(z_i' \gamma\right)}{\partial x_s} = -\gamma_s z'\gamma \cdot \phi\left(z_i' \gamma\right)
$$
Также
$$
\frac{\partial \Phi\left(-z_i' \gamma\right)}{\partial x_s} = -\gamma_s \cdot \phi\left(-z_i' \gamma\right)=
-\gamma_s \cdot \phi\left(z_i' \gamma\right)
$$

Тогда
$$
\frac{\partial\lambda_2}{\partial x_s} = \frac{-\gamma_s z'\gamma \cdot \phi\left(z_i' \gamma\right)\cdot\Phi\left(-z_i' \gamma\right) + \phi\left(z_i' \gamma\right)\cdot \gamma_s \cdot \phi\left(z_i' \gamma\right)}
{\Phi\left(-z_i' \gamma\right)^2} = - \gamma_s\left( z'\gamma \cdot \lambda_2 - \lambda_2^2 \right)
$$

Значит 
$$
\frac{\partial E(y_i^*|z_i = 0)}{\partial x_s} =
\beta_s + \rho \sigma(z) \cdot  \gamma_s\left( z'\gamma \cdot \lambda_2 - \lambda_2^2 \right) \approx 0.0185
$$

In [35]:
%%R

age_ME2 <- coef_mle["age"] + rho_mle * sigma_mle *
                                (work_li * lambda_est_2 -
                                 lambda_est_2 ^ 2) *
                                coef_s_est["age"]
age_ME2

       age 
0.01851122 


## Часть 4. Модель Ньюи.
$\bf4.1.^{*}$ Опишите преимущества и недостатки метода Ньюи по сравнению с методом Хекмана.

$\bf{\text{Решение:}}$

Так как метод Ньюи является полупараметрическим, для его использования нет необходомости в допущении о многомерном нормальном распределении случайных ошибок: даже при других распределениях его оценки будут состоятельными (в отличие от метода Хекмана, но, конечно, не факт, что самыми эффективными). Еще одним преимуществом метода является то, что его достаточно просто реализовать.

Недостатками подхода является то, что он довольно вычислительно сложен (используется бутстрап и leave-one-out-cross-validation), константа в основном уравнении не идентифицируется, метод (как и двухшаговый Хекман) требует наличия exclusion restriction, а также не позволяет оценить параметры распределения ошибок, из-за чего невозможно получить прогнозы условного математического ожидания целевой переменной для случая, когда $z_i=0$. 

Основные аргументы взяты [отсюда](https://www.hse.ru/data/xf/127/606/1158/Потанин_диссертация.pdf).

$\bf4.2.^{***}$ Взяв за основу любую модель бинарного выбора (для простоты можно и параметрическую, например, логит), произвольную сглаживающую функцию и используя leave-one-out кросс-валидацию для подбора степени полинома, оцените модель Ньюи, описав осуществленные для ее построения шаги. Результат представьте в форме таблицы, содержащей оценки и бутстрапированные стандартные ошибки. Сравните оценки модели Ньюи и моделей Хекмана, основанных на двухшаговой процедуре и ММП.

$\bf{\text{Решение:}}$

1. Используем логит-модель для оценки уравнения участия:

In [36]:
import statsmodels.formula.api as smf

logit = smf.logit("lfp ~ kids + age", data=df).fit()
logit.summary()

Optimization terminated successfully.
         Current function value: 0.670768
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    lfp   No. Observations:                  753
Model:                          Logit   Df Residuals:                      750
Method:                           MLE   Df Model:                            2
Date:                 вс, 04 дек 2022   Pseudo R-squ.:                 0.01900
Time:                        19:25:25   Log-Likelihood:                -505.09
converged:                       True   LL-Null:                       -514.87
Covariance Type:            nonrobust   LLR p-value:                 5.629e-05
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.4188      0.533      4.538      0.000       1.374       3.463
kids          -0.2279      0.061     -3.766      0.000      -0.346      -0.109
age           -0.0417      0.011     -3.800      0.000      -0.063      -0.020
==============================================================================
"""

2. Оценим линейный индекс и посчитаем значение сглаживающей функции $f(x)=\frac{x^2}{e^x}$:

In [37]:
s = logit.predict() ** 2 / np.exp(logit.predict())

3. С помощью leave-one-out кросс-валидации и MSE определим лучшую степень полинома.

Напишем функцию для подсчета MSE при полиноме k-ой степени:

In [38]:
from tqdm.auto import tqdm
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression

def calc_MSE(k, df=df, s=s):
    X = df[['age', 'educ']].copy()
    y = df['wage']
    all_mse = []
    
    # добавляем колонки степеней в X
    for i in range(k):
        X[str(i+1)] = s ** (i+1)
        
    # leave-one-out
    loo = LeaveOneOut()
    
    for train_index, test_index in loo.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        linear_reg = LinearRegression().fit(X_train, y_train)
        all_mse.append(mse(y_test, linear_reg.predict(X_test)))
        
    return np.mean(all_mse)


In [39]:
ks = np.arange(11)

MSEs = []
for k in tqdm(ks):
    MSEs.append(calc_MSE(k))

  0%|          | 0/11 [00:00<?, ?it/s]

In [40]:
pd.DataFrame(zip(np.arange(11), MSEs), columns=['k', 'MSE'])

,k,MSE
0,0,9.513720
1,1,9.456847
2,2,9.427377
3,3,9.441177
4,4,9.469032
5,5,9.496864
6,6,9.523703
7,7,9.561589
8,8,9.608631
9,9,9.603718


Лучшим k оказалось 2.

In [41]:
np.set_printoptions(suppress=True)

X = df[['age', 'educ']].copy()
y = df['wage']
    
# добавляем колонки степеней в X
for i in range(2):
    X[str(i+1)] = s ** (i+1)
    
linear_reg = LinearRegression().fit(X, y)
linear_reg.coef_

array([  0.03421941,   0.44118663, -36.70907691, 132.00506969])

Теперь, объединим все шаги в один пайплайн и бутстрапом найдем соответствующие оценки:

In [42]:
def get_one_iter(df, max_k=10):
    ### Бутстрапированная выборка
    df_boot = df.sample(n=df.shape[0], replace=True)
        
    ### Logit
    logit = smf.logit("lfp ~ kids + age", data=df_boot).fit(disp=0)
    s = logit.predict() ** 2 / np.exp(logit.predict())
    
    ### LOO
    ks = np.arange(max_k + 1)

    MSEs = []
    for k in ks:
        MSEs.append(calc_MSE(k, df=df_boot, s=s))
            
    best_k = np.argmin(MSEs)
        
    ### Линейная модель
    X = df_boot[['age', 'educ']].copy()
    y = df_boot['wage']
    
    # добавляем колонки степеней в X
    for i in range(best_k):
        X[str(i+1)] = s ** (i+1)
    
    linear_reg = LinearRegression().fit(X, y)
    return [linear_reg.intercept_] + list(linear_reg.coef_[:2]) + [best_k]
      

In [43]:
def get_results(n_iter, max_k=3):
    coefs = []
    for i in tqdm(range(n_iter)):
        coefs.append(get_one_iter(df, max_k=max_k))
    return coefs


In [44]:
np.random.seed(999)

ests = get_results(100, max_k=5)

  0%|          | 0/100 [00:00<?, ?it/s]

Средняя степень полинома на итерациях: 

In [45]:
np.mean(np.array(ests)[:, -1])

2.31

Результаты получились в сравнении с моделями Хекмана следующими:

In [46]:

betas = ['intercept', 'age', 'educ']
tobit = [-7.02410, -0.01668,  0.69445]
tobit_se = [1.51982, 0.02344, 0.08370]
heckman_mle = [-2.9391697, 0.0220963, 0.4982631]
mle_se = [1.23099, 0.02005, 0.06584]
heckman_2step = [-2.74649310, 0.02586941, 0.49643489]
step2_se = [1.33958, 0.02259, 0.06601]

newey = np.mean(ests, axis=0)[:-1]
newey_se = np.sqrt(np.var(ests, axis=0)[:-1] * len(ests) / (len(ests) - 1))


pd.DataFrame(zip(betas, tobit, tobit_se, heckman_mle, 
                 mle_se, heckman_2step, step2_se,
                 newey, newey_se), columns=['Beta', 'Tobit', 'Tobit s.e.', 'Heckman 1', 'Heckman 1 s.e.',
                                            'Heckman 2', 'Heckman 2 s.e.', 'Newey', 'Newey s.e.'])

,Beta,Tobit,Tobit s.e.,Heckman 1,Heckman 1 s.e.,Heckman 2,Heckman 2 s.e.,Newey,Newey s.e.
0,intercept,-7.02410,1.51982,-2.939170,1.23099,-2.746493,1.33958,-8.291322,34.846848
1,age,-0.01668,0.02344,0.022096,0.02005,0.025869,0.02259,0.029476,0.017269
2,educ,0.69445,0.08370,0.498263,0.06584,0.496435,0.06601,0.437257,0.056971


В таблице выше под Heckman 1 и 2 обозначены, соответственно, модели Хекмана с ММП и двухшаговой процедурой. После колонки коэффициента для каждой модели следует колонка с его стандартным отклонением. Оценки коэффициентов и стандартных отклонений в методе Ньюи получены с помощью бутстрапа.

Оценки как коэффициентов, так и стандартных отклонений получились похожими (несмотря на то, что мы взяли выдуманную из головы сглаживающую функцию). Стоит отметить, что оценка константы при использовании метода Ньюи сильно зависит от степени полинома, ее стандартное отклонение во много раз превосходит аналоги других подходов, что согласуется с теорией: константа в методе Ньюи не оперделяется однозначно.